# Knapsack example

- Maximize the total value of the items in the knapsack
- Stay within the capacity of the knapsack
- Each item has a value, weight, and a volume
- The knapsack has both weight and volume capacity

In [ ]:
values = [1, 2, 3, 4]
weights = [2, 2, 3, 3]
volumes = [3, 3, 2, 2]
n = 4
variables = list(range(n))
max_weight = 4
max_volume = 6

penalty_strength_weight = 20
penalty_strength_volume = 20

# Create the BQM model
Decision variable is binary variable $x_i$ for each item $i$. If $x_i=1$, then item $i$ is in the knapsack, zero otherwise.

The coefficient of the variable is the negative of its value to account for the fact that we minimize the objective.


x_1 | x_2 | x_3 | x_4 | total value | total weight | total volume | feasible|
:----:|:---:|:----:|:----:|:----:|:-----:|:-----:|:------:
1   | 0   |0   | 0   | 1  | 2  | 3  |   True
0   | 1   |0   | 0   | 2  | 2  | 3  |   True
0   | 0   |1   | 0   | 3  | 3  | 2  |   True
**0**   | **0**   |**0**   | **1**   | **4**  | **3**  | **2**  |   **True**
0   | 0   |1   | 1   | 7  | 6  | 4  |   False
1   | 1   |0   | 0   | 3  | 4  | 6  |   True

In [ ]:
from dimod.binary import BinaryQuadraticModel

bqm = BinaryQuadraticModel('BINARY')

variables = [bqm.add_variable(f'x_{v}', -values[v]) for v in variables]

Add the capacity constraint as a linear inequality term.

In [ ]:
slacks_weight = bqm.add_linear_inequality_constraint(
    [(x, w) for x, w in zip(variables, weights)],
    constant=-max_weight,
    lagrange_multiplier=penalty_strength_weight,
    label='capacity'
)

Similarly we define a set of slack variables for the volume capacity constraint.

In [ ]:
slacks_volume = bqm.add_linear_inequality_constraint(
    [(x, v) for x, v in zip(variables, volumes)],
    constant=-max_volume,
    lagrange_multiplier=penalty_strength_volume,
    label='volume_capacity'
)

For a small problem with less than 20 items, we can solve the QUBO exactly by enumerating all solutions.

In [ ]:
from dimod import ExactSolver

response = ExactSolver().sample(bqm)
samples = response.samples()
best_solution = response.first.sample
print(response.truncate(5))

Let's take a closer look at the solution.

- The main variables specify the knapsack assginment.
- The first set of slack variables show that the total weight of the selected items is 1 unit less than the weight capacity of the knapsack
- The second set of slack variables show that the total volume of the selected items is 1 unit less than the volume capacity of the knapsack

In [ ]:
print({k: v for k, v in best_solution.items() if k in variables})
print({k: v for k, v in best_solution.items() if k in {v for v, _ in slacks_weight}})
print({k: v for k, v in best_solution.items() if k in {v for v, _ in slacks_volume}})

In [ ]:
def violations(sample, slack_variables, array, capacity):
    slack_total_weight = 0
    weight = 0
    for i, (v, c) in enumerate(slack_variables):
        slack_total_weight += sample[v] * c
    for i, v in enumerate(variables):
        weight += array[i] * sample[v]
    print('Total of items: ', weight)
    print('Total slack value: ', slack_total_weight)
    print('Left hand side: ', weight + slack_total_weight)
    print('Satisfied? ', weight + slack_total_weight <= capacity)
    

In [ ]:
print('Weight capacity constraint:')
violations(samples[0], slacks_weight, weights, max_weight)
print('Volume capacity constraint:')
violations(samples[0], slacks_volume, volumes, max_volume)